# SOS 2021 installation check

In [23]:
COLAB=True # if running in COLAB
#COLAB=False # if not running on COLAB
GPU=True # if using GPU (should be True on COLAB and possibly Linux)
#GPU=False # no GPU on Mac and possibly Linux



## Enabling and testing the GPU

(largely copied from https://colab.research.google.com/notebooks/gpu.ipynb)

On COLAB only, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down


In [24]:
if COLAB: 
    # Display available and used memory.
    !free -h
    print("-"*70)
    # Display the CPU specification.
    !lscpu
    print("-"*70)
    # Display the GPU specification (if available).
    !(nvidia-smi | grep -q "has failed") && echo "No GPU found!" || nvidia-smi


              total        used        free      shared  buff/cache   available
Mem:            12G        2.9G        3.8G         13M        6.0G         10G
Swap:            0B          0B          0B
----------------------------------------------------------------------
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse3

## Check XGBoost

In [25]:
import xgboost as xgb
print(xgb.__version__)  # preinstalled version 0.9.0 20210123
!pip install xgboost --upgrade # install 1.3.3 20210123 
print(xgb.__version__)


1.3.3
Requirement already up-to-date: xgboost in /usr/local/lib/python3.6/dist-packages (1.3.3)
1.3.3


In [26]:
from sklearn.datasets import load_boston
boston = load_boston()
import pandas as pd

data = pd.DataFrame(boston.data)
data.columns = boston.feature_names
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [27]:
X, y = data.iloc[:,:-1],data.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

In [28]:
from sklearn import datasets
import xgboost as xgb
import numpy as np
import time

iris = datasets.load_iris()
X = iris.data
y = iris.target
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations
starting_time = time.time( )
model = xgb.train(param, D_train, steps)

training_time = time.time( ) - starting_time
print("Training time:",training_time)
from sklearn.metrics import accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))


[11:37:23] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Training time: 0.008787870407104492
Accuracy = 0.9666666666666667


## Check lightgbm


In [29]:
## Check lightgbm
import lightgbm as lgb
print (lgb.__version__) # Tested with 2.3.1
!pip install lightgbm --upgrade
print (lgb.__version__)

3.1.1
Requirement already up-to-date: lightgbm in /usr/local/lib/python3.6/dist-packages (3.1.1)
3.1.1


In [30]:
gbm = lgb.LGBMClassifier()


starting_time = time.time( )

gbm.fit(X_train, Y_train)

training_time = time.time( ) - starting_time
print("Training time:",training_time)

Training time: 0.029570817947387695


# Check tensorflow 

In [31]:
if not COLAB:
    !pip install tensorflow

In [32]:
import tensorflow as tf
print (tf.__version__)  # COLAB default 20210123 2.4.0


2.4.0


In [33]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  if GPU:
        raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
if GPU:
    gpu()

# Run the op several times.
from datetime import datetime
print (datetime.now())
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

2021-01-24 11:37:25.795127
Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.862845119000042
GPU (s):
0.03868061600002193
GPU speedup over CPU: 74x
